# Exploring starts Monte Carlo Control
An implementation of MC control through "exploring starts"

The gridworld has the shape(3,4) with a winning state "w"(0,3), and a lossing state "l"(1,3), a non valid state "x"(2,1) and a start state s(3,0)

|  |  |  |  |
|---|---|---|---|
|  |  |  | w |
|  |  |  | l |
|  | x |  |  |
| s |  |  |  |

### This is a simple not optimal policy evaluation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import grid_world

### Define the maximum change before finish the loop

In [2]:
CONVERGENCE_THRESHOLD = 10e-5
GAMMA = 0.9

### Auxiliary function to display the values of a policy after finishing iterative policy evaluation

In [3]:
def print_values(V,grid):
    for i in range(grid.width):
        print("--------------------------")
        for j in range(grid.height):
            v = V.get((i,j),0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")

### Auxiliary function to display a stochastic policy

In [4]:
def print_policy(P,grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            a = P.get((i,j),' ')
            if isinstance(a,dict):
                a = list(a)[0]
            print("  %s  |" % a, end="")
        print("")

### From or defined grid world file, import a negative grid ,retrieve all actions and states and print grid rewards
Negative grid is used to encourage the agent to find a shortest path to the goal

In [5]:
grid = grid_world.Grid.negative_grid(step_cost = -0.1)
states = grid.all_states()
actions = list(set([action   for action_tup in grid.actions.values() for action in action_tup]))

In [6]:
def argmax_dict(dictionary):
    # returns the argmax key and the max value from a dictionary
    # will be used for policy improvement from Q
    max_key = None
    max_val = float("-inf")
    
    for k,v in dictionary.items():
        if v > max_val:
            max_val = v
            max_key = k
            
    return max_key,max_val
        
argmax_dict({"a":1,"b":2})

('b', 2)

In [7]:
actions

['L', 'R', 'U', 'D']

In [8]:
def play_game(grid,policy,gamma=0.9,max_steps = 20):
    # exploring starts
    start_states = list(grid.actions.keys())
    start_index = np.random.choice(len(start_states))
    grid.set_state(start_states[start_index])
    
    s = grid.current_state()
    a = np.random.choice(actions)
    
    states_actions_and_rewards = [(s,a,0)] 
    game_steps = 0 #to control infinite loops
    # play a game until game is finished and record all new states and rewards
    while not grid.game_over():
        
        old_s = grid.current_state()
        r = grid.move(a)
        s = grid.current_state()
        
        # if agent bumps into a wall, the episode ends with negative reward 
        # to make it avoid this state,action in the future
        if old_s == s :
            states_actions_and_rewards.append((s,None,-100))
            break
        elif  game_steps > max_steps:
            break
        elif grid.game_over():
            states_actions_and_rewards.append((s,None,r))
            break
        else:
            a = policy[s]
            states_actions_and_rewards.append((s,a,r))
            
        game_steps += 1
        
    # calculate every state returns by going backwards
    states_and_actions_returns = []
    
    G=0
    for s,a,r in reversed(states_actions_and_rewards):
        
        G = r +GAMMA*G
        states_and_actions_returns.append((s,a,G))
        
    return states_and_actions_returns

In [9]:
grid = grid_world.Grid.negative_grid()

In [10]:
print("Rewards of grid")
print_values(grid.rewards,grid)

Rewards of grid
--------------------------
-0.10|-0.10|-0.10| 1.00|
--------------------------
-0.10| 0.00|-0.10|-1.00|
--------------------------
-0.10|-0.10|-0.10|-0.10|


### Initialize random policy and Q(s,a)

In [11]:
policy = {}

for s in grid.actions.keys():
    policy[s] = np.random.choice(actions)
print_policy(policy,grid)

---------------------------
  L  |  L  |  R  |     |
---------------------------
  U  |     |  L  |     |
---------------------------
  R  |  D  |  D  |  D  |


In [12]:
play_game(grid,policy,GAMMA)

[((1, 3), None, -1.0), ((2, 3), 'U', -0.9)]

In [13]:
Q = defaultdict(lambda: defaultdict(lambda:0))
Q[(0,0)][0]

0

In [14]:
def MS_exploring_starts(grid,policy,episodes,gamma):
    Q = defaultdict(lambda: defaultdict(lambda:0))
    state_action_visit_count = defaultdict(lambda: defaultdict(lambda:1))
    
    for i in range(episodes):
        
        states_actions_and_returns = play_game(grid,policy,gamma)
        visited_states_actions = set()
        
        #  policy evaluation
        for s,a,G in states_actions_and_returns:
            
            if (s,a) not in visited_states_actions:
                
                # incremental average of return calculation
                Q[s][a] = Q[s][a] + (1.0/state_action_visit_count[s][a])*(G-state_action_visit_count[s][a])
                visited_states_actions.add((s,a))
                state_action_visit_count[s][a] += 1

        
            
                
            #policy improvement
        for s in states:
            state_argmax = argmax_dict(Q[s])
            if not state_argmax[0] is None:
                policy[s] = state_argmax[0]
    return policy,Q
        
policy,Q = MS_exploring_starts(grid,policy,10000,GAMMA)

In [15]:
policy

{(0, 0): 'R',
 (0, 1): 'R',
 (0, 2): 'R',
 (1, 0): 'U',
 (1, 2): 'R',
 (2, 0): 'U',
 (2, 1): 'U',
 (2, 2): 'L',
 (2, 3): 'U'}

In [16]:
print("Policy")
print_policy(policy,grid)

Policy
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  U  |  L  |  U  |


## Conclusions
* ES created infinite loops for this gridgworld in cases where the agent tries things like right in a state, left in the next, can explore-exploit epsilon greedy solve this?
* Same as all monte carlo, no model of env dynamics are needed